#### Author: Alexis Geslin

In [1]:
import re
import time
import glob
import pandas as pd
import numpy as np
import argparse
from statistics import stdev
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/cours/cs224n/project/LLM-Prop/


/content/drive/MyDrive/cours/cs224n/project/LLM-Prop


In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('-'*20)
    print(f'I have {torch.cuda.device_count()} devices, currently on {torch.cuda.current_device()}')
    print('-'*20)
else:
    print('-'*20)
    print("You are running on CPU only")
    print('-'*20)
    device = torch.device("cpu")

--------------------
I have 1 devices, currently on 0
--------------------


In [5]:
train_labels_data = pd.read_csv(f'./embeddings/pred_labels_train_train15000_200epoch_125098.csv')
valid_labels_data = pd.read_csv(f'./embeddings/pred_labels_valid_train15000_200epoch_9945.csv')
test_labels_data = pd.read_csv(f'./embeddings/pred_labels_test_train15000_200epoch_11531.csv')

train_embeddings  = pd.read_csv(f'./embeddings/e5small_train_125098.csv', header=None)
valid_embeddings  = pd.read_csv(f'./embeddings/e5small_valid_9945.csv', header=None)
test_embeddings  = pd.read_csv(f'./embeddings/e5small_test_11531.csv', header=None)


In [6]:
#concat embeddings and numerical tokens and make it a tensor
X_train = torch.tensor(train_embeddings.values).float().to(device)
X_valid = torch.tensor(valid_embeddings.values).float().to(device)
X_test = torch.tensor(test_embeddings.values).float().to(device)


Y_train = torch.tensor(train_labels_data.label.values).float().reshape(-1,1).to(device)
Y_valid = torch.tensor(valid_labels_data.label.values).float().reshape(-1,1).to(device)
Y_test = torch.tensor(test_labels_data.label.values).float().reshape(-1,1).to(device)

#print shapes
print(X_train.shape, Y_train.shape,X_train.device,Y_train.device)
print(X_valid.shape, Y_valid.shape,X_valid.device,Y_valid.device)
print(X_test.shape, Y_test.shape,X_test.device,Y_test.device)

torch.Size([125098, 384]) torch.Size([125098, 1]) cuda:0 cuda:0
torch.Size([9945, 384]) torch.Size([9945, 1]) cuda:0 cuda:0
torch.Size([11531, 384]) torch.Size([11531, 1]) cuda:0 cuda:0


In [7]:
dim_embeddings = train_embeddings.shape[1]
mae_loss_function = nn.L1Loss()
mseloss = nn.MSELoss()


In [8]:
X_train_mini = X_train[:15000]
Y_train_mini = Y_train[:15000]
bs = 256 if torch.cuda.is_available() else 32
print(bs)

train_dataset = torch.utils.data.TensorDataset(X_train, Y_train)
train_dataset_mini = torch.utils.data.TensorDataset(X_train_mini, Y_train_mini)
valid_dataset = torch.utils.data.TensorDataset(X_valid, Y_valid)
test_dataset = torch.utils.data.TensorDataset(X_test, Y_test)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=False)
train_dataloader_mini = torch.utils.data.DataLoader(train_dataset_mini, batch_size=bs, shuffle=False)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=bs, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=False)

256


In [9]:
class perceptronHead(nn.Module):
    def __init__(self, embeddings_dim,mydroprate = 0.1):
        super(perceptronHead, self).__init__()

        self.MLP_embedding = nn.Sequential(
            nn.Dropout(mydroprate),
            nn.Linear(embeddings_dim, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )
        # initialize weights with Kaiming
        for m in self.MLP_embedding:
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                print(f'initialized {m}')



    def forward(self, X):
        x_out = self.MLP_embedding(X)
        return x_out

def init_weights(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)

class perceptronHead2(nn.Module):
    def __init__(self, embeddings_dim,mydroprate = 0.1):
        super(perceptronHead2, self).__init__()

        self.MLP_embedding = nn.Sequential(
            nn.Dropout(mydroprate),
            nn.Linear(embeddings_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )
        # initialize weights with Kaiming
        for m in self.MLP_embedding:
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                print(f'initialized {m}')



    def forward(self, X):
        x_out = self.MLP_embedding(X)
        return x_out

class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.fc = nn.Linear(in_features, in_features)
        self.bn = nn.BatchNorm1d(in_features)

    def forward(self, x):
        return nn.functional.relu(self.bn(self.fc(x)) + x)

class perceptronHead3(nn.Module):
    def __init__(self, embeddings_dim,mydroprate = 0.1):
        super(perceptronHead3, self).__init__()

        self.MLP_embedding = nn.Sequential(
            nn.Dropout(mydroprate),
            nn.Linear(embeddings_dim, 512),
            nn.ReLU(),
            ResidualBlock(512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            ResidualBlock(128),
            nn.ReLU(),
            nn.Linear(128, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
            # nn.LayerNorm(1)
        )
        # initialize weights with Kaiming
        for m in self.MLP_embedding:
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                print(f'initialized {m}')



    def forward(self, X):
        x_out = self.MLP_embedding(X)
        return x_out

def init_weights(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)

In [ ]:
model = perceptronHead3(dim_embeddings,mydroprate=0.1)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0015)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
my_training_loader = train_dataloader_mini
my_training_loader = train_dataloader

initialized Linear(in_features=384, out_features=512, bias=True)
initialized Linear(in_features=512, out_features=128, bias=True)
initialized Linear(in_features=128, out_features=8, bias=True)
initialized Linear(in_features=8, out_features=1, bias=True)


In [ ]:
epochs =100

best_valid_loss = 1000000000
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for i, (x, y) in enumerate(my_training_loader):
        optimizer.zero_grad()
        y_pred = model(x)
        loss = mae_loss_function(y_pred, y)
        train_loss += loss.item()
        before_update = {name: param.clone().detach() for name, param in model.named_parameters()}
        loss.backward()
        # for name, param in model.named_parameters():
        #     if param.grad is not None and torch.all(param.grad == 0):
        #         print(f"All-zero gradient for {name}")
        optimizer.step()

        # for name, param in model.named_parameters():
        #     if torch.equal(before_update[name], param):
        #         print(f"No update for {name}")
        # if i % 10000 == 0:
    train_loss /= len(my_training_loader)
    print(f'Epoch {epoch}, Training Loss {round((train_loss),3)}')




    model.eval()

    valid_preds = []
    valid_labels = []
    with torch.no_grad():
        for i, (x, y) in enumerate(valid_dataloader):
            y_pred = model(x)
            valid_preds.append(y_pred)
            # valid_preds.append(y_pred.detach().cpu())
            valid_labels.append(y)
    valid_preds = torch.cat(valid_preds)
    valid_labels = torch.cat(valid_labels)
    valid_loss = mae_loss_function(valid_preds, valid_labels)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './AG_checkpoints/best_model.pth')
        print("Saving a new best model")
    scheduler.step(best_valid_loss)
    print(f'Epoch {epoch}, Validation Loss {valid_loss.item()}')

Epoch 0, Training Loss 0.447
Saving a new best model
Epoch 0, Validation Loss 0.4691212773323059
Epoch 1, Training Loss 0.447
Epoch 1, Validation Loss 0.4704788625240326
Epoch 2, Training Loss 0.445
Epoch 2, Validation Loss 0.47003501653671265
Epoch 3, Training Loss 0.444
Epoch 3, Validation Loss 0.47051846981048584
Epoch 4, Training Loss 0.447
Epoch 4, Validation Loss 0.46941906213760376
Epoch 5, Training Loss 0.446
Saving a new best model
Epoch 5, Validation Loss 0.46905431151390076
Epoch 6, Training Loss 0.445
Saving a new best model
Epoch 6, Validation Loss 0.46778684854507446
Epoch 7, Training Loss 0.445
Epoch 7, Validation Loss 0.4691285490989685
Epoch 8, Training Loss 0.443
Epoch 8, Validation Loss 0.46860066056251526
Epoch 9, Training Loss 0.445
Epoch 9, Validation Loss 0.46857643127441406
Epoch 10, Training Loss 0.444
Epoch 10, Validation Loss 0.4688628315925598
Epoch 11, Training Loss 0.447
Epoch 11, Validation Loss 0.46888241171836853
Epoch 12, Training Loss 0.447
Epoch 12, 

In [ ]:
best_model = perceptronHead3(dim_embeddings)
best_model.to(device)
best_model.load_state_dict(torch.load('./AG_checkpoints/best_model.pth'))
# best_model = model

best_model.eval()
with torch.no_grad():
    test_preds = []
    test_labels = []
    for i, (x, y) in enumerate(test_dataloader):
        y_pred = best_model(x)
        test_preds.append(y_pred)
        test_labels.append(y)
    test_preds = torch.cat(test_preds)
    test_labels = torch.cat(test_labels)
    test_loss = mae_loss_function(test_preds, test_labels)
    print(f'Test Loss {test_loss.item()}')

initialized Linear(in_features=384, out_features=512, bias=True)
initialized Linear(in_features=512, out_features=128, bias=True)
initialized Linear(in_features=128, out_features=8, bias=True)
initialized Linear(in_features=8, out_features=1, bias=True)
Test Loss 0.4777367413043976


<ipython-input-94-5b5bc6db7301>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load('./AG_checkpoints/best_model.pth'))


In [ ]:
# prompt: # get the device best_model and x are on

print(f"Device for best_model and x: {device}")


Device for best_model and x: cuda


In [ ]:
print(next(best_model.parameters()).device)

cuda:0


Training with scheduler, for 200 epochs,  lr=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

### TESTING BEST MODEL

In [10]:
#model loading
best_model = perceptronHead3(dim_embeddings)
best_model.to(device)
best_model.load_state_dict(torch.load(f'./AG_checkpoints/best_model_006_best_e5small.pth'))


best_model.train()
#train
with torch.no_grad():
    train_preds = []
    train_labels = []
    for i, (x, y) in enumerate(train_dataloader):
        y_pred = best_model(x)
        train_preds.append(y_pred)
        train_labels.append(y)
    train_preds = torch.cat(train_preds)
    train_labels = torch.cat(train_labels)
    train_loss = mae_loss_function(train_preds, train_labels)
    print(f'Training Loss {round(train_loss.item(),3)}')

#valid
best_model.eval()
with torch.no_grad():
    valid_preds = []
    valid_labels = []
    for i, (x, y) in enumerate(valid_dataloader):
        y_pred = best_model(x)
        valid_preds.append(y_pred)
        valid_labels.append(y)
    valid_preds = torch.cat(valid_preds)
    valid_labels = torch.cat(valid_labels)
    valid_loss = mae_loss_function(valid_preds, valid_labels)
    print(f'Validation Loss {round(valid_loss.item(),3)}')

#train
with torch.no_grad():
    test_preds = []
    test_labels = []
    for i, (x, y) in enumerate(test_dataloader):
        y_pred = best_model(x)
        test_preds.append(y_pred)
        test_labels.append(y)
    test_preds = torch.cat(test_preds)
    test_labels = torch.cat(test_labels)
    test_loss = mae_loss_function(test_preds, test_labels)
    print(f'Test Loss {round(test_loss.item(),3)}')

initialized Linear(in_features=384, out_features=512, bias=True)
initialized Linear(in_features=512, out_features=128, bias=True)
initialized Linear(in_features=128, out_features=8, bias=True)
initialized Linear(in_features=8, out_features=1, bias=True)


<ipython-input-10-8c0dec72f89a>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(f'./AG_checkpoints/best_model_006_best_e5small.pth'))


Training Loss 0.442
Validation Loss 0.468
Test Loss 0.478
